In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Check this notebook for getting started
https://www.kaggle.com/kaushikholla/0-752-eda-baseline-model-svm



In the above notebook, I have approached the problem as a classification problem and in this notebook I will look at this problem as regression problem and see how the model performs.

In [ ]:
## Importing the core libraries
import pandas as pd
import numpy as np
import scipy

## Sklearn packages for modelling
from sklearn.linear_model import Ridge
# from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
## Changing the setting to display entire column
pd.set_option('display.max_columns', None)

## Note:
In order to make prediction on the given dataset, we will train our model on dataset from previous jigsaw competation and use that model to score the sentences given in this challenge.

In [ ]:
## Reading the data
df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
df.head()

In [ ]:
## Since this challenge is about rating the severity of the toxicity, we will give more weightage to sever_toxic column
df['severe_toxic'] = df['severe_toxic']*2.5

## Lets have our y variable as sum of all the toxicity
df['y'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis = 1).astype(int)

In [ ]:
## Getting rid of all other columns except out text and y
df = df[['comment_text', 'y']]
## taking a look at first 5 columns
df.head()

In [ ]:
## Having a look at random 5 samples from the dataset
df.sample(5)

In [ ]:
## Lets have a look at the distribution of y variable
df.hist(column = "y")

We can see that we have a number values between 0 and 6(inclusive) with a lot of values being 0. Lets take a look at value count to get better idea

In [ ]:
## Checking out the values of the distribution
df['y'].value_counts()

If we go ahead and build a model on the about distribution, the model will overfit to 0's as the value of 0 is significantly higher than others. Lets sample the data so that the model doesnt overfit. 

In [ ]:
## Sampling the data
df = pd.concat([df[df['y'] > 0], df[df['y'] == 0].sample(int(len(df[df['y'] > 0])*1.5))], axis = 0).sample(frac = 1)

In [ ]:
## Lets have a look at the distribution of y variable
df.hist(column = "y")

print(df['y'].value_counts())

## Building a Regression model using Ridge Regression

In [ ]:
## tf-idf
## Converting text to number format to feed it as input to the ML model
vectorizer = TfidfVectorizer(analyzer = 'char_wb', max_df = 0.5, min_df = 3, ngram_range = (3,6))
X = vectorizer.fit_transform(df['comment_text'])

## Looking at the shape of the created data
X.shape

In [ ]:
## Model
## trying ridge regression model
model = Ridge()
model.fit(X, df['y'])

## Comments to Score

In [ ]:
## Reading the comments that we need to score
df_ = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

In [ ]:
df_.head()

In [ ]:
## Extracting the text and converting it for the model 
X_ = vectorizer.transform(df_['text'])
pred = model.predict(X_)

In [ ]:
## Adding prediction back to submission file
df_['score'] =  pred

In [ ]:
df_.head()

In [ ]:
df_[['comment_id', 'score']].to_csv("submission.csv", index=False)

## Reference
https://www.kaggle.com/julian3833/jigsaw-incredibly-simple-naive-bayes-0-768
https://www.kaggle.com/steubk/jrsotc-ridgeregression